# TF-IDF

We'll rely on term frequency times inverted document frequency ([TF-IDF](https://web.stanford.edu/~jurafsky/slp3/11.pdf)), a powerful implementation of the [bag-of-words model](https://web.stanford.edu/~jurafsky/slp3/B.pdf), to measure meaningful similarity between documents while disregarding word order. Let's start by generating a matrix for the separate constituent parts of _Stjórn_.

In [1]:
import os,glob,json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def normalize(target):
    # This dict limits orthographical variation beyond the rule sets
    # of stjorn-extract.ipynb and menota-extract.ipynb:
    matrix = {
        'j': 'i',
        'v': 'u',
        'ð': 'þ',
        'á': 'a',
        'ǽ': 'æ',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'ý': 'y',
        'ǿ': 'ø',
        'k': 'c', # rather than vice versa, because of Latin (e.g. Lucifer)
        '[': '',
        ']': ''
        }
    for k,v in matrix.items():
        target = target.replace(k, v)
    return target

titles = ['prologue', 'introduction', 'gn', 'ex', 'lv', 'nm', 'dt', 'ios', 'idc', 'rt', '1sm', '2sm', '3rg', '4rg']
tokens = []
for title in titles:
    with open(f"nlp/{title}.txt") as raw:
        document = raw.read().replace('\n', ' ')
        tokens.extend(document.split())

work_indices = {
    'stjorn1': (650,124417),
    'stjorn2': (124417,147678),
    'stjorn3': (147678,156943,160719),
    'stjorn4': (156943,160719)
}

stjorn = dict()
for _work, _range in work_indices.items():
    if len(_range) == 2:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]]))
    else:
        stjorn[_work] = normalize(' '.join(tokens[_range[0]:_range[1]] + tokens[_range[2]:]))

menota = dict()
for text in glob.glob('../menota/dipl/*txt'):
    ref = os.path.basename(text).replace('.txt', '')
    with open(text) as doc:
        # We'll subject Menota to the same normalization standard as Stjórn:
        menota[ref] = normalize(doc.read().replace('\n', ''))

Note the arguments passed to the vectorizer class below. `min_df` sets a cutoff for the minimum number of documents in which a term has to appear in order to be included in the model. Since terms exclusive to single compositions are among the things that interest us, we'll leave this at `1`. `max_df` sets a cutoff point above which relative document frequency a term is ignored: in other words, a value of `0.8` ignores words occurring in over 80 percent of documents. Changing this value massively changes the document similarity scores downstream, and drastic changes in the setting have a pronounced effect on the document similarity rankings as well. A strict threshold is at any rate required to gain an insight into the relevance of individual terms, as leaving `max_df` at its default of `1.0` would lead the model to conclude that "oc" is the most meaningful term in many of our documents, while "æigi" is the top-ranking term for _Stjórn III_ as well as the _Norwegian Homily Book_ with a value as low as `0.4`. We can therefore leave the score high in the _Stjórn_-internal comparison, but will ramp it up significantly for the larger corpus.

In [3]:
vectorizer = TfidfVectorizer(min_df=1, max_df=0.8)
model = vectorizer.fit_transform(stjorn.values())
df = pd.DataFrame(cosine_similarity(model), stjorn.keys(), stjorn.keys())
df

,stjorn1,stjorn2,stjorn3,stjorn4
stjorn1,1.000000,0.063975,0.510216,0.405235
stjorn2,0.063975,1.000000,0.054947,0.057555
stjorn3,0.510216,0.054947,1.000000,0.334973
stjorn4,0.405235,0.057555,0.334973,1.000000


After eliminating such variation as vowel length marks and the þ/ð distinction, these are now all pretty similar to one another, with the biggest difference between _Stjórn II_ and _III_.

Now let's first add _Konungs skuggsjá_ from Menota, as well as Unger's own edition of the _Norwegian Homily Book_. Fingers crossed that we have got the normalization standard of the former to approach Unger's methods reasonably well.

In [4]:
# We want only those parts of Unger's NHB matched in Menota:
nhb_titles = ['alcuin', 'hom', 'olafr', 'visio', 'paternoster', 'anhang1']
nhb = ''
for title in nhb_titles:
    filepath = f'../nhb/nlp/{title}.txt'
    with open(filepath) as doc:
        nhb = nhb + normalize(doc.read().replace('\n', ''))
stjorn_plus = []
for v in stjorn.values():
    stjorn_plus.append(v)
stjorn_plus.extend([menota['nks235g_konungs_skuggsja'], nhb])
model = vectorizer.fit_transform(stjorn_plus)
df = pd.DataFrame(cosine_similarity(model), list(stjorn.keys()) + ['ks', 'nhb'], list(stjorn.keys()) + ['ks', 'nhb'])
df

,stjorn1,stjorn2,stjorn3,stjorn4,ks,nhb
stjorn1,1.000000,0.308887,0.211326,0.370354,0.010920,0.045486
stjorn2,0.308887,1.000000,0.145243,0.358827,0.067233,0.022487
stjorn3,0.211326,0.145243,1.000000,0.209011,0.219577,0.414135
stjorn4,0.370354,0.358827,0.209011,1.000000,0.000887,0.056356
ks,0.010920,0.067233,0.219577,0.000887,1.000000,0.229910
nhb,0.045486,0.022487,0.414135,0.056356,0.229910,1.000000


_Stjórn III_ and _Konungs skuggsjá_ share material cognate within the vernacular, so this connection standing out as the strongest between _Konungs skuggsjá_ and the constituent parts of _Stjórn_ comes as no surprise; if anything, the difference in match with the other parts is rather small. In fact, the _Norwegian Homily Book_ has a higher match with _Stjórn III_ than _Konungs skuggsjá_ does, which may be explained at least in part by the closer subject match for those parts of _Stjórn III_ not reflected in _Konungs skuggsjá_.

Next, let's model all of Menota along with Stjórn. Perhaps we'll leave Unger's _Homily Book_ in alongside the Menota edition, just for comparison's sake.

In [5]:
vectorizer = TfidfVectorizer(min_df=1, max_df=0.3)
corpus = []
titles = []
for k,v in stjorn.items():
    titles.append(k)
    corpus.append(v)
titles.append('nhb')
corpus.append(nhb)
for k,v in menota.items():
    titles.append(k)
    corpus.append(v)
model = vectorizer.fit_transform(corpus)
df = pd.DataFrame(cosine_similarity(model), titles, titles)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df

,stjorn1,stjorn2,stjorn3,stjorn4,nhb,nraNorrFragm75_kross_saga,am132_egils_saga,am162btheta_njals_saga,nraNorrFragm64_barlaams_saga,nraNorrFragm81A_benedikts_regla,am1056IX_konungs_skuggsja_fragment,am78_kristinrettir,am63_heimskringla3,dg4-7_strengleikar,am132_droplaugasona_saga,am132_kormaks_saga,nraNorrFragm72x76_dialogar,nraNorrFragm53_haralds_saga_hardrada,am132_finnboga_saga,nraNorrFragm70_agotu_saga,nraNorrFragm62_karlamagnuss_saga,nraNorrFragm60A_stjorn,am132_fostbraedra_saga,lbsFragm82_olafs_saga_helga,nraNorrFragm58B_konungs_skuggsja,nraNorrFragm60C_stjorn,holmPerg30_landslog,am619_norwegian_homily_book,nraNorrFragm57_jons_saga_helga,nraNorrFragm69_nikulass_saga,am56_landslog,wolfAug9-10_egils_saga,nraNorrFragm66_thomass_saga,holmPerg17_thomass_saga,am383I_thorlaks_saga,holmPerg4_thidreks_saga,am132_njals_saga,am36_heimskringla2,am544_voluspa,am162bkappa_njals_saga,am305_landslog,nraNorrFragm58C_konungs_skuggsja,am132_olkofra_thattr,konungs_skuggsja_am243ba,nraNorrFragm54_sverris_saga,nraNorrFragm55B_hakonar_saga,nraNorrFragm79_mariu_saga,gks2365_voluspa,am243balpha_konungs_skuggsja,nraNorrFragm51_fagrskinna,am132_viga-glums_saga,am279a_gragas,am677_gregory,am132_laxdoela_saga,am302_landslog,am178_thidreks_saga,nraNorrFragm81B_benedikts_regla,am132_bandamanna_saga,nraNorrFragm71_gregors_saga_pafa,am655_laeknisbok,am519a_alexanders_saga,holmPerg34_landslog,am162balpha_njals_saga,am113_islendingabok,nraNorrFragm7_landslog,nraNorrFragm67_thomass_saga,nraNorrFragm56_thorgils_saga,nks235g_konungs_skuggsja,am132_hallfredar_saga,am35_heimskringla1,am242_codex_wormianus,nraNorrFragm78_mariu_saga,konungs_skuggsja_fragment_am1056xi,dg8II_olafs_saga,nraNorrFragm80_pals_saga,nraNorrFragm63_karlamagnuss_saga,nraNorrFragm77_dialogar,am28_codex_runicus,holmPerg34_boejarlog,dg8I_landslog,nraNorrFragm60B_stjorn,nraNorrFragm55A_hakonar_saga,skbA120_marys_complaint,nraNorrFragm59_rimbegla,nraNorrFragm65_floress_saga,nraNorrFragm52_olafs_saga_helga_hin_elzta,holmPerg6_barlaams_saga,nraNorrFragm68_brendanuss_saga,nraNorrFragm61_karlamagnuss_saga,nraNorrFragm58A_konungs_skuggsja
stjorn1,1.000000,0.351157,0.135833,0.366428,0.020519,0.005083,0.015877,0.014361,0.010414,0.001665,0.008127,0.004449,0.010518,0.012868,0.011211,0.006438,0.006167,0.007730,0.008024,0.002316,0.040858,0.053051,0.011683,0.008389,0.138412,0.018450,0.007152,0.020006,0.028391,0.029020,0.005465,0.021205,0.027054,0.069657,0.012091,0.018686,0.014278,0.011975,0.028858,0.042633,0.004617,0.006276,0.005288,0.012543,0.110711,0.018615,0.022700,0.009580,0.012543,0.001030,0.011566,0.003635,0.020817,0.018560,0.003624,0.010737,0.003175,0.007790,0.007550,0.008131,0.020117,0.005143,0.042064,0.002455,0.005374,0.020027,0.004964,0.002690,0.007010,0.010118,0.050540,0.018758,0.008127,0.005844,0.048790,0.094705,0.021637,0.005845,0.002852,0.028313,0.033178,0.015386,0.005787,0.032327,0.023317,0.007409,0.029892,0.005572,0.008066,0.013983
stjorn2,0.351157,1.000000,0.175287,0.380052,0.018765,0.002950,0.034831,0.009470,0.006318,0.000885,0.000809,0.003316,0.031357,0.035597,0.051547,0.023944,0.004301,0.004014,0.027619,0.001942,0.035894,0.001639,0.045568,0.009356,0.182295,0.009422,0.007500,0.020437,0.009761,0.018741,0.003529,0.047841,0.012619,0.004573,0.009634,0.026891,0.083970,0.010190,0.027247,0.105942,0.003289,0.004139,0.053022,0.017802,0.128600,0.013253,0.048546,0.006564,0.017802,0.000335,0.047980,0.001995,0.036978,0.062093,0.002399,0.071142,0.002793,0.038364,0.005095,0.007223,0.010443,0.107760,0.106531,0.001500,0.001820,0.023736,0.003425,0.004709,0.051314,0.009167,0.034146,0.033746,0.000809,0.004295,0.032933,0.138324,0.023348,0.001629,0.086159,0.062109,0.032243,0.009456,0.000369,0.058540,0.031056,0.004709,0.015406,0.002741,0.007217,0.007330
stjorn3,0.135833,0.175287,1.000000,0.205968,0.077587,0.007280,0.053655,0.012998,0.025273,0.006202,0.015863,0.023444,0.039861,0.095112,0.057566,0.029265,0.014365,0.010310,0.036830,0.002960,0.052294,0.038329,0.047910,0.034551,0.109810,0.146830,0.015525,0.077556,

The score for the two editions of the _Norwegian Homily Book_ may serve as our proof of method: with any `max_df` setting, these come to a similarity in the range 0.93~0.97. As this compares an edition of Unger's with a Menota transcription, as does our comparison of _Stjórn_ with the remainder of the Menota corpus, we may be confident that the scores give a fair indication of lexical similarity to the extent the TF-IDF measure can provide one.

Let's take the constituent parts of _Stjórn_ one at a time and rank the Menota material by similarity. At this point, having demonstrated our method, we may take Unger's _Homily Book_ out:

In [6]:
nhb_index = titles.index('nhb')
corpus.pop(nhb_index)
titles.pop(nhb_index)
model = vectorizer.fit_transform(corpus)
df = pd.DataFrame(cosine_similarity(model), titles, titles)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

def rank(subject):
    sorted_df = df.sort_values([subject], ascending=False)
    print(sorted_df[subject][:10].to_string()) # Limiting output for brevity

In [23]:
for i in work_indices.keys():
    print(f"{i}:")
    print("--------------------------------------------")
    rank('stjorn1')
    print('')

stjorn1:
--------------------------------------------
stjorn1                             1.000000
stjorn4                             0.366097
stjorn2                             0.350482
nraNorrFragm58B_konungs_skuggsja    0.138099
stjorn3                             0.135325
nraNorrFragm54_sverris_saga         0.110196
nraNorrFragm63_karlamagnuss_saga    0.094376
holmPerg17_thomass_saga             0.068821
nraNorrFragm60A_stjorn              0.052779
am242_codex_wormianus               0.050475

stjorn2:
--------------------------------------------
stjorn1                             1.000000
stjorn4                             0.366097
stjorn2                             0.350482
nraNorrFragm58B_konungs_skuggsja    0.138099
stjorn3                             0.135325
nraNorrFragm54_sverris_saga         0.110196
nraNorrFragm63_karlamagnuss_saga    0.094376
holmPerg17_thomass_saga             0.068821
nraNorrFragm60A_stjorn              0.052779
am242_codex_wormianus               

TODO: rewrite findings after adjusting `max_df`

Our next step is to investigate which word forms are conspicuously associated with the constituent parts of _Stjórn_. The outcome of this query is even more greatly affected by our `max_df` setting.

In [24]:
scores = pd.DataFrame(model.toarray(), titles, vectorizer.get_feature_names_out())
for i in work_indices.keys():
    print(f"{i}:")
    print("------------------------")
    print(scores.loc[i].sort_values(ascending=False)[:20])
    print('')

stjorn1:
------------------------
medr          0.613277
aa            0.370005
edr           0.268476
gud           0.201136
þiat          0.197219
meþr          0.112242
þers          0.102301
scylld        0.101643
guds          0.101441
taladi        0.096613
sagdi         0.095440
uaru          0.093442
scolastica    0.090924
uidr          0.084131
sealfs        0.083970
ioseph        0.083465
iacob         0.074603
israels       0.069756
moyses        0.066933
stad          0.060845
Name: stjorn1, dtype: float64

stjorn2:
------------------------
aa          0.463507
þier        0.311013
gud         0.284710
moyses      0.269591
med         0.252225
suo         0.212032
mællti      0.200452
aaron       0.170140
drottinn    0.156811
cyni        0.121595
brutt       0.117494
balaam      0.109792
ydr         0.108907
moysen      0.106027
baud        0.089268
drottins    0.086806
lyd         0.082078
madr        0.078369
israel      0.076854
moysi       0.076771
Name: stjorn2, dtype:

After discarding commonly spelled function words using the `max_df` setting, _Stjórn I_ stands out for its exceptional _spelling_ of a function word: "medr" for "meðr" (the latter normalized to "meþr" for the present analysis); in fact, it stands out by writing &lt;d&gt; for &lt;ð&gt; quite generally. This is a peculiarity of Unger's manuscript, [AM 226](https://handrit.is/manuscript/view/da/AM02-0226/0?iabr=on#mode/2up), specifically, and not found (or not as pronounced?) in [AM 227](https://handrit.is/manuscript/view/is/AM02-0227/0?iabr=on#mode/2up). This of course reduces the lexical profile of _Stjórn I_ largely to an orthographical profile unless we level the playing field by normalizing &lt;ð&gt; to &lt;d&gt; throughout the corpus. CHECK THAT MENOTA TRANSCRIBERS HAVE FAITHFULLY COPIED D WHERE APPLICABLE. However, orthographical information is information, too: we may note that although this spelling is rare in Menota, the Holm. Perg. 17 manuscript of _Thómass saga erkibyskups_ has it three times, while the AM 132 text of _Laxdœla saga_ and the Norr. Fragm. 58B fragment of _Konungs skuggsjá_ have it once each. 